In [2]:
from dotenv import load_dotenv
load_dotenv(override=True) 

True

#### Define a Local LLM and Tools

In [13]:
from langchain_ollama import ChatOllama
from langchain.tools import tool
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

@tool
def add_numbers(a: int, b: int):
    "Add 2 numbers and returns result"
    return a + b
@tool
def subtract_numbers(a: int, b: int):
    "subtracts 2 numbers and returns result"
    return a - b
@tool
def multiply_numbers(a: int, b: int):
    "Multiply 2 numbers and returns result"
    return a * b
wikipedia_tool = WikipediaQueryRun(api_wrapper= WikipediaAPIWrapper())

tools = [add_numbers, subtract_numbers, multiply_numbers, wikipedia_tool]

llama_chat = ChatOllama(base_url="http//localhost:11434",
                        model="qwen3:8b",
                        reasoning= True,
                        temperature=0.5,
                        max_tokens=250)

#### Define Agent

In [4]:
from langchain.agents import create_agent

agent = create_agent(model= llama_chat, 
                     tools=tools,
                     system_prompt="You are a helpful assistant that can answer questions using provided tools. Always structure your final response in Json format.")

#### Invoke Agent using HumanMessage

In [4]:
from langchain.messages import HumanMessage
from langchain.agents import create_agent

agent = create_agent(model= llama_chat, 
                     tools=tools,
                     system_prompt="You are a helpful assistant that can answer questions using provided tools. Always structure your final response in Json format.")
response = agent.invoke({"messages": [HumanMessage("What is sum of 2 and 4. Who won 2025 Women's Cricket World Cup")]})
print(response["messages"])
print(response["messages"][0].content)
print(response["messages"][-1].content)

[HumanMessage(content="What is sum of 2 and 4. Who won 2025 Women's Cricket World Cup", additional_kwargs={}, response_metadata={}, id='a3b2819f-fa53-4779-ad3c-736c325d7cf1'), AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'qwen2.5:latest', 'created_at': '2025-12-17T15:41:42.228488Z', 'done': True, 'done_reason': 'stop', 'total_duration': 12431498542, 'load_duration': 4924610959, 'prompt_eval_count': 377, 'prompt_eval_duration': 4211596375, 'eval_count': 54, 'eval_duration': 2911978122, 'logprobs': None, 'model_name': 'qwen2.5:latest', 'model_provider': 'ollama'}, id='lc_run--019b2cf9-3b2e-7e22-8a44-3c50c6813845-0', tool_calls=[{'name': 'add_numbers', 'args': {'a': 2, 'b': 4}, 'id': '17e406b8-2e72-4c26-81da-586b79688e38', 'type': 'tool_call'}, {'name': 'wikipedia', 'args': {'query': "2025 Women's Cricket World Cup"}, 'id': '72c8bbc1-d8bf-4812-8351-9500e65a31fd', 'type': 'tool_call'}], usage_metadata={'input_tokens': 377, 'output_tokens': 54, 'total_tokens': 431

#### Invoke Agent Using ChatPromptTemplate

In [14]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import create_agent
agent = create_agent(model= llama_chat, tools=tools)
prompt_template = ChatPromptTemplate([
    ("system", "You are an expert in Math and latest news across the globe"),
    ("user", "Whats sum of 2 and 5 multiplied by 7?"),
    ("user", "Who won Women Cricket World cup in 2025"),
    ("user", "Give me both the answers in JSON format")
])
result = agent.invoke({"messages":prompt_template.format_messages()})
print(result["messages"][-1].content)

{
  "math_result": 49,
  "cricket_winner": "India"
}


#### Log Tool Calling and Reasoning and Response using content_blocks 

In [15]:
for message in result["messages"]:
    for block in message.content_blocks:
        if block["type"] == "text":
            print("Text:", block["text"])
        elif block["type"] == "tool_call":
            print("Tools Called:", block["name"])
        elif block["type"] == "reasoning":
            print("Reasoning:", block["reasoning"])

Text: You are an expert in Math and latest news across the globe
Text: Whats sum of 2 and 5 multiplied by 7?
Text: Who won Women Cricket World cup in 2025
Text: Give me both the answers in JSON format
Reasoning: Okay, let's tackle the user's questions. First, they want the sum of 2 and 5 multiplied by 7. Hmm, I need to parse that correctly. The phrase is "sum of 2 and 5 multiplied by 7." So, does that mean (2 + 5) * 7 or 2 + (5 * 7)? The wording says "sum of 2 and 5" which is 7, then multiplied by 7. So it's (2+5)*7. That would be 7*7=49. Let me check if the functions available can handle that. The user provided functions for add_numbers, subtract, multiply. So first add 2 and 5, then multiply by 7. So first call add_numbers with a=2, b=5. Then take that result and multiply by 7 using multiply_numbers.

Next question: Who won the Women's Cricket World Cup in 2025? Wait, the 2025 Women's Cricket World Cup hasn't happened yet because the current year is 2023. The last tournament was in 2